# 사용자 쿼리 기반 매장 추천

## 00. Setting

In [ ]:
import pandas as pd
import numpy as np
import faiss
import torch

import json
import zipfile
import io
import glob
import os

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.ticker as ticker
import seaborn as sns

from sentence_transformers import SentenceTransformer, CrossEncoder
from sklearn.metrics import precision_score

In [2]:
# 폰트 설정

font_path = '/Users/kangahhyun/Library/Fonts/NanumSquareRoundOTFR.otf'
font_name = fm.FontProperties(fname=font_path).get_name()
plt.rcParams['font.family'] = font_name
plt.rcParams['axes.unicode_minus'] = False

## 01. Data Set

1.1. 데이터 출처
- 소상공인시장진흥공단_상가(상권)정보 (출처. 공공데이터포털)

1.2. 데이터 규모
- 전체 데이터 수: 2,695,924개

In [ ]:
# 데이터 로드 (압축 파일 해제 및 통합)
data_dir = './data'
zip_files = glob.glob(os.path.join(data_dir, '*.zip'))
zip_path = zip_files[0]

data_list = []
with zipfile.ZipFile(zip_path, 'r') as zf:
    for file_name in zf.namelist():
        if file_name.lower().endswith('.csv'):
            with zf.open(file_name) as f:
                try:
                    df_tmp = pd.read_csv(f, encoding='utf-8')
                except:
                    df_tmp = pd.read_csv(f, encoding='cp949')
                data_list.append(df_tmp)

df = pd.concat(data_list, ignore_index=True)

# 필요한 컬럼만 선별 및 결측치 제거
df = df[['상호명', '상권업종대분류명', '상권업종소분류명', '시도명', '시군구명', '도로명주소']].dropna()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2695924 entries, 0 to 2695924
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   상호명       object
 1   상권업종대분류명  object
 2   상권업종소분류명  object
 3   시도명       object
 4   시군구명      object
 5   도로명주소     object
dtypes: object(6)
memory usage: 144.0+ MB


In [ ]:
#df.to_csv('./data/store_data.csv', index=False, encoding='utf-8-sig')

## 02. Base Line: bi-encoder semantic search

In [ ]:
df = pd.read_csv('./data/store_data.csv', encoding='utf-8-sig')

### 2.1. 데이터 선별
- 전체 데이터를 다 사용할 수 없기 때문에 50,000개 data sampling

In [ ]:
SAMPLE_SIZE = 50000

df_sampled = df.sample(
    n=SAMPLE_SIZE,
    random_state=42
).reset_index(drop=True)


### 2.2. 검색용 문서 생성
- `build_store_document`로 각 행을 자연어 문장으로 변환한 데이터(`documents`) 생성

In [8]:
def build_store_document(row):
    return (
        f"가게 이름은 {row['상호명']}입니다. "
        f"{row['시도명']} {row['시군구명']}에 위치한 "
        f"{row['상권업종대분류명']} 업종 중 "
        f"{row['상권업종소분류명']} 매장입니다. "
        f"주소는 {row['도로명주소']}입니다."
    )


documents = df_sampled.apply(build_store_document, axis=1).tolist()


### 2.3. Embedding

2.3.1. 디바이스
- mps 사용

2.3.2. SentenceTransformer 임베딩 모델 선택
- 한국어 포함 다국어 지원
- semantic search에 특화

In [9]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
embedder = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    device=device
)
corpus_embeddings = embedder.encode(
    documents,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True
)

Batches: 100%|██████████| 3125/3125 [01:27<00:00, 35.57it/s]


### 2.4. FAISS Index

In [10]:
def build_faiss_index(embeddings: np.ndarray):
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim) # Inner Product
    faiss.normalize_L2(corpus_embeddings) # cosine similarity
    index.add(embeddings)
    return index

In [11]:
index = build_faiss_index(corpus_embeddings)

### 2.5. Evaluation용 데이터 생성

In [12]:
EVAL_SIZE = 500

df_eval = df_sampled.sample(
    n=EVAL_SIZE,
    random_state=42
).reset_index(drop=True)

def generate_query(row):
    return (
        f"{row['시군구명']}에 있는 "
        f"{row['상권업종소분류명']} 업종 매장을 찾아줘"
    )

eval_dataset = []

for _, row in df_eval.iterrows():
    eval_dataset.append({
        "query": generate_query(row),
        "positive_doc_id": df_sampled.index[
            (df_sampled['상호명'] == row['상호명']) &
            (df_sampled['도로명주소'] == row['도로명주소'])
        ][0]
    })

In [13]:
eval_dataset[:3]

[{'query': '천안시 서북구에 있는 사업시설 유지·관리 서비스업 업종 매장을 찾아줘', 'positive_doc_id': 33553},
 {'query': '목포시에 있는 장난감 소매업 업종 매장을 찾아줘', 'positive_doc_id': 9427},
 {'query': '남양주시에 있는 기타 교육지원 서비스업 업종 매장을 찾아줘', 'positive_doc_id': 199}]

### 2.6. Semantic search (+reranker)

In [14]:
def semantic_search(query, top_k=10):
    query_embedding = embedder.encode(
        query,
        convert_to_numpy=True
    ).reshape(1, -1)

    scores, indices = index.search(query_embedding, top_k)
    return indices[0].tolist()

In [15]:
reranker = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    device=device
)

In [16]:
def semantic_search_with_reranker(query, top_k=10):
    candidate_ids = semantic_search(query, top_k=top_k)

    candidate_docs = [documents[i] for i in candidate_ids]

    pairs = [(query, doc) for doc in candidate_docs]
    rerank_scores = reranker.predict(pairs)

    reranked = sorted(
        zip(candidate_ids, rerank_scores),
        key=lambda x: x[1],
        reverse=True
    )

    reranked_ids = [doc_id for doc_id, _ in reranked]
    return reranked_ids


In [17]:
def evaluate_model(eval_dataset, search_fn, top_k=10):
    recall_scores = []
    mrr_scores = []

    for item in eval_dataset:
        query = item['query']
        positive_id = item['positive_doc_id']

        retrieved_ids = search_fn(query, top_k=top_k)

        # Recall@K
        recall = 1 if positive_id in retrieved_ids else 0
        recall_scores.append(recall)

        # MRR
        if positive_id in retrieved_ids:
            rank = retrieved_ids.index(positive_id) + 1
            mrr_scores.append(1.0 / rank)
        else:
            mrr_scores.append(0.0)

    return {
        "Recall@K": np.mean(recall_scores),
        "MRR": np.mean(mrr_scores)
    }


### 2.7. Result

In [19]:
TOP_K = 10

baseline_result = evaluate_model(
    eval_dataset,
    search_fn=semantic_search,
    top_k=TOP_K
)

reranker_result = evaluate_model(
    eval_dataset,
    search_fn=semantic_search_with_reranker,
    top_k=TOP_K
)

print("[Baseline: Semantic Search]")
for k, v in baseline_result.items():
    print(f"{k}: {v:.4f}")

print("\n[Improved: Semantic Search + Reranker]")
for k, v in reranker_result.items():
    print(f"{k}: {v:.4f}")


[Baseline: Semantic Search]
Recall@K: 0.0180
MRR: 0.0042

[Improved: Semantic Search + Reranker]
Recall@K: 0.0180
MRR: 0.0096


In [20]:
results_df = pd.DataFrame([
    {
        "Model": "Semantic Search",
        "Recall@K": baseline_result["Recall@K"],
        "MRR": baseline_result["MRR"]
    },
    {
        "Model": "Semantic Search + Reranker",
        "Recall@K": reranker_result["Recall@K"],
        "MRR": reranker_result["MRR"]
    }
])

results_df


,Model,Recall@K,MRR
0,Semantic Search,0.018,0.004222
1,Semantic Search + Reranker,0.018,0.009619


In [21]:
def print_search_result(query, top_k=5):
    print(f"QUERY: {query}")
    print("=" * 80)

    # Baseline
    print("[Baseline: Semantic Search]")
    baseline_ids = semantic_search(query, top_k=top_k)

    for rank, doc_id in enumerate(baseline_ids, start=1):
        print(f"\n#{rank}")
        print(documents[doc_id])

    # Reranker
    print("" + "=" * 80)
    print("[Add reranker: Semantic Search + Reranker]")
    reranked_ids = semantic_search_with_reranker(query, top_k=top_k)

    for rank, doc_id in enumerate(reranked_ids, start=1):
        print(f"\n#{rank}")
        print(documents[doc_id])


In [22]:
test_query = "강남구에 있는 카페를 찾아줘"
print_search_result(test_query, top_k=5)


QUERY: 강남구에 있는 카페를 찾아줘
[Baseline: Semantic Search]

#1
가게 이름은 금성장여관입니다. 강원특별자치도 강릉시에 위치한 숙박 업종 중 여관/모텔 매장입니다. 주소는 강원특별자치도 강릉시 옥가로58번길 9-1입니다.

#2
가게 이름은 강남성균검도관입니다. 서울특별시 강남구에 위치한 교육 업종 중 입시·교과학원 매장입니다. 주소는 서울특별시 강남구 논현로 523입니다.

#3
가게 이름은 흥덕한국체대태권스쿨태권도장입니다. 경기도 용인시 기흥구에 위치한 교육 업종 중 태권도/무술학원 매장입니다. 주소는 경기도 용인시 기흥구 흥덕1로79번길 2입니다.

#4
가게 이름은 최강달인춘천닭갈비.막국수입니다. 서울특별시 강남구에 위치한 음식 업종 중 닭/오리고기 구이/찜 매장입니다. 주소는 서울특별시 강남구 선릉로 657입니다.

#5
가게 이름은 영진장어입니다. 강원특별자치도 강릉시에 위치한 음식 업종 중 해산물 구이/찜 매장입니다. 주소는 강원특별자치도 강릉시 연곡면 해안로 1365입니다.
[Add reranker: Semantic Search + Reranker]

#1
가게 이름은 최강달인춘천닭갈비.막국수입니다. 서울특별시 강남구에 위치한 음식 업종 중 닭/오리고기 구이/찜 매장입니다. 주소는 서울특별시 강남구 선릉로 657입니다.

#2
가게 이름은 영진장어입니다. 강원특별자치도 강릉시에 위치한 음식 업종 중 해산물 구이/찜 매장입니다. 주소는 강원특별자치도 강릉시 연곡면 해안로 1365입니다.

#3
가게 이름은 흥덕한국체대태권스쿨태권도장입니다. 경기도 용인시 기흥구에 위치한 교육 업종 중 태권도/무술학원 매장입니다. 주소는 경기도 용인시 기흥구 흥덕1로79번길 2입니다.

#4
가게 이름은 금성장여관입니다. 강원특별자치도 강릉시에 위치한 숙박 업종 중 여관/모텔 매장입니다. 주소는 강원특별자치도 강릉시 옥가로58번길 9-1입니다.

#5
가게 이름은 강남성균검도관입니다. 서울특별시 강남구에 위치한 교육 업종 중 입시·교과

결과를 확인했을 때, 데이터를 제대로 가져오고 있지 못하는 것 같다. 개선 필요.

## 03. 검색 프로세스 재설계

In [25]:
df = pd.read_csv('./data/store_data.csv', encoding='utf-8-sig')
df = df.sample(n=50000, random_state=42).reset_index(drop=True)


### 3.1. Embedding 모델 변경

In [28]:
embedder = SentenceTransformer(
    "snunlp/KR-SBERT-V40K-klueNLI-augSTS",
    device=device
)

### 3.2. corpus 재설계

In [27]:
def build_document(row):
    return f"""
    [업종] {row['상권업종소분류명']}
    [지역] {row['시도명']} {row['시군구명']}
    [가게명] {row['상호명']}
    [주소] {row['도로명주소']}
    """

documents = df.apply(build_document, axis=1).tolist()


### 3.3. 규칙 기반 쿼리 파싱

In [29]:
def parse_query(query):
    conditions = {"region": None, "category": None}

    if "강남구" in query:
        conditions["region"] = "강남구"

    if "카페" in query:
        conditions["category"] = "카페"

    return conditions

### 3.4. 구조적 필터링

In [30]:
def prefilter(df, conditions):
    filtered = df.copy()

    if conditions["region"]:
        filtered = filtered[
            filtered["시군구명"].str.contains(
                conditions["region"], case=False, regex=False, na=False
            )
        ]

    if conditions["category"]:
        filtered = filtered[
            filtered["상권업종소분류명"].str.contains(
                conditions["category"], case=False, regex=False, na=False
            )
        ]

    return filtered.reset_index(drop=True)


### 3.5. corpus embedding

In [31]:
corpus_embeddings = embedder.encode(
    documents,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)


Batches: 100%|██████████| 3125/3125 [03:11<00:00, 16.30it/s]


### 3.6. FAISS Index

In [32]:
index = build_faiss_index(corpus_embeddings)

### 3.7. Search Function

In [33]:
def search(query, top_k=5):
    # 1. Query 파싱
    conditions = parse_query(query)

    # 2. 구조적 필터링
    filtered_df = prefilter(df, conditions)

    if len(filtered_df) == 0:
        return []

    filtered_docs = filtered_df.apply(build_document, axis=1).tolist()

    # 3. Embedding
    query_embedding = embedder.encode(
        query,
        normalize_embeddings=True
    )

    # 4. FAISS 검색
    filtered_embeddings = embedder.encode(
        filtered_docs,
        batch_size=16,
        normalize_embeddings=True,
        convert_to_numpy=True
    )

    temp_index = faiss.IndexFlatIP(filtered_embeddings.shape[1])
    temp_index.add(filtered_embeddings)

    scores, indices = temp_index.search(
        np.array([query_embedding]), top_k * 5
    )

    candidates = [
        filtered_docs[i] for i in indices[0]
    ]

    # 5. Rerank
    pairs = [[query, doc] for doc in candidates]
    rerank_scores = reranker.predict(pairs)

    reranked = sorted(
        zip(candidates, rerank_scores),
        key=lambda x: x[1],
        reverse=True
    )

    return reranked[:top_k]


In [34]:
query = "강남구에 있는 카페를 찾아줘"
results = search(query)

for i, (doc, score) in enumerate(results, 1):
    print(f"#{i} (score={score:.4f})")
    print(doc)


#1 (score=8.8772)

    [업종] 독서실/스터디 카페
    [지역] 서울특별시 강남구
    [가게명] 시작스터디카페대치
    [주소] 서울특별시 강남구 도곡로 442
    
#2 (score=8.8042)

    [업종] 카페
    [지역] 서울특별시 강남구
    [가게명] 매머드익스프레스청담점
    [주소] 서울특별시 강남구 압구정로79길 38
    
#3 (score=8.7929)

    [업종] 카페
    [지역] 서울특별시 강남구
    [가게명] 프라텔리
    [주소] 서울특별시 강남구 봉은사로4길 31
    
#4 (score=8.7890)

    [업종] 카페
    [지역] 서울특별시 강남구
    [가게명] 이디야커피청담역점
    [주소] 서울특별시 강남구 삼성로 721
    
#5 (score=8.7770)

    [업종] 카페
    [지역] 서울특별시 강남구
    [가게명] 메가엠지씨커피압구정카페골목점
    [주소] 서울특별시 강남구 선릉로 815
    


### 3.8. Evaluation

In [35]:
def is_relevant_cafe(doc: str) -> bool:
    """
    doc 문자열이 '강남구 + 카페' 조건을 만족하는지 판단
    """
    doc = doc.lower()
    return (
        ("강남" in doc or "강남구" in doc)
        and ("카페" in doc or "커피" in doc)
    )

In [ ]:
def label_results(results):
    """
    search(query) 결과를 [0,1] relevance 리스트로 변환
    """
    labels = []
    for doc, _ in results:
        labels.append(1 if is_relevant_cafe(doc) else 0)
    return labels


In [37]:
def precision_at_k(labels, k):
    labels = labels[:k]
    if len(labels) == 0:
        return 0.0
    return sum(labels) / k


def recall_at_k(labels, total_relevant, k):
    if total_relevant == 0:
        return 0.0
    return sum(labels[:k]) / total_relevant

def mrr_score(labels):
    for idx, label in enumerate(labels):
        if label == 1:
            return 1.0 / (idx + 1)
    return 0.0


In [38]:
def evaluate_search(query, search_fn, k=5):
    results = search_fn(query)
    labels = label_results(results)

    total_relevant = sum(
        is_relevant_cafe(doc) for doc in df["document"]
    )

    metrics = {
        "Precision@K": precision_at_k(labels, k),
        "Recall@K": recall_at_k(labels, total_relevant, k),
        "MRR": mrr_score(labels),
    }

    return metrics, results, labels


### 3.9. Result

In [39]:
metrics, results, labels = evaluate_search(query, search, k=5)

print("\n=== Evaluation Result ===")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

print("\n=== Search Results ===")
for i, ((doc, score), label) in enumerate(zip(results, labels), 1):
    print(f"#{i} | score={score:.4f} | relevant={label}")
    print(doc)



=== Evaluation Result ===
Precision@K: 1.0000
Recall@K: 0.0833
MRR: 1.0000

=== Search Results ===
#1 | score=8.8772 | relevant=1

    [업종] 독서실/스터디 카페
    [지역] 서울특별시 강남구
    [가게명] 시작스터디카페대치
    [주소] 서울특별시 강남구 도곡로 442
    
#2 | score=8.8042 | relevant=1

    [업종] 카페
    [지역] 서울특별시 강남구
    [가게명] 매머드익스프레스청담점
    [주소] 서울특별시 강남구 압구정로79길 38
    
#3 | score=8.7929 | relevant=1

    [업종] 카페
    [지역] 서울특별시 강남구
    [가게명] 프라텔리
    [주소] 서울특별시 강남구 봉은사로4길 31
    
#4 | score=8.7890 | relevant=1

    [업종] 카페
    [지역] 서울특별시 강남구
    [가게명] 이디야커피청담역점
    [주소] 서울특별시 강남구 삼성로 721
    
#5 | score=8.7770 | relevant=1

    [업종] 카페
    [지역] 서울특별시 강남구
    [가게명] 메가엠지씨커피압구정카페골목점
    [주소] 서울특별시 강남구 선릉로 815
    


## 04. "강남구 카페" 찾는 쿼리말고 다른 쿼리의 결과가 잘나오도록 하려면..?

### 4.1. 구조적 필터링

4.1.1. 지역명 정규화

In [40]:
df['시도명'].value_counts()

시도명
경기도        12094
서울특별시       9863
경상남도        3119
부산광역시       2886
경상북도        2689
인천광역시       2493
충청남도        2256
대구광역시       2121
강원특별자치도     1993
전라남도        1881
전북특별자치도     1836
충청북도        1642
대전광역시       1430
광주광역시       1427
울산광역시       1009
제주특별자치도     1009
세종특별자치시      252
Name: count, dtype: int64

In [ ]:
REGION_KEYWORDS = {
    "서울특별시": ["서울", "서울시", "서울특별시"],
    "경기도": ["경기", "경기도"],
    "부산광역시": ["부산", "부산시", "부산광역시"],
    "대구광역시": ["대구", "대구시", "대구광역시"],
    "인천광역시": ["인천", "인천시", "인천광역시"],
    "광주광역시": ["광주", "광주시", "광주광역시"],
    "대전광역시": ["대전", "대전시", "대전광역시"],
    "울산광역시": ["울산", "울산시", "울산광역시"],
    "세종특별자치시": ["세종", "세종시", "세종특별자치시"],
    "강원특별자치도": ["강원", "강원도", "강원특별자치도"],
    "충청북도": ["충북", "충청북도"],
    "충청남도": ["충남", "충청남도"],
    "전북특별자치도": ["전북", "전라북도", "전북특별자치도"],
    "전라남도": ["전남", "전라남도"],
    "경상북도": ["경북", "경상북도"],
    "경상남도": ["경남", "경상남도"],
    "제주특별자치도": ["제주", "제주도", "제주특별자치도"],
}


4.1.2. 업종명 정규화

In [42]:
df["상권업종소분류명"].value_counts().head(20)

상권업종소분류명
백반/한정식        3155
카페            2229
미용실           2169
부동산 중개/대리업    2045
입시·교과학원       1298
슈퍼마켓          1109
요리 주점         1070
경영 컨설팅업       1006
돼지고기 구이/찜      971
편의점            962
기타 의류 소매업      960
김밥/만두/분식       837
치킨             735
펜션             709
자동차 정비소        700
피부 관리실         657
여성 의류 소매업      652
국/탕/찌개류        550
빵/도넛           528
노래방            494
Name: count, dtype: int64

In [44]:
# 지역명과 달리 업종명은 종류가 많아서 일부만 선정
CATEGORY_KEYWORDS = {
    "카페": ["카페", "커피", "디저트", "브런치"],
    "해산물": ["해산물", "횟집", "해물탕", "수산", "생선"],
    "술집": ["호프", "포장마차", "주점", "이자카야"],
    "한식": ["한식", "백반", "국밥", "찌개"],
    "숙박": ["호텔", "콘도", "여관", "모텔", "펜션"]
}

### 4.2. Search

In [63]:
def parse_query(query):
    conditions = {
        "regions": [], "categories": []
    }

    for region_name, keywords in REGION_KEYWORDS.items():
        if any(kw in query for kw in keywords):
            conditions["regions"].append(region_name)

    for cat, keywords in CATEGORY_KEYWORDS.items():
        for kw in keywords:
            if kw in query:
                conditions["categories"].append(cat)
                break

    return conditions

In [46]:
def filter_dataframe(df, regions=None, categories=None):
    filtered = df.copy()

    if regions:
        filtered = filtered[filtered["시도명"].isin(regions)]

    if categories:
        all_keywords = []
        for cat in categories:
            keywords = CATEGORY_KEYWORDS.get(cat, [cat]) # 사전에 없으면 입력값 그대로 사용
            all_keywords.extend(keywords)
        
        pattern = "|".join(set(all_keywords))
        
        filtered = filtered[
            filtered["상권업종대분류명"].str.contains(pattern, na=False) |
            filtered["상권업종소분류명"].str.contains(pattern, na=False)
        ]

    return filtered.reset_index(drop=True)

In [47]:
def search(query, df_filtered, embedder, index, k=5):
    if len(df_filtered) == 0:
        return []

    query_emb = embedder.encode(
        [query],
        normalize_embeddings=True
    )

    scores, indices = index.search(
        query_emb,
        min(k, len(df_filtered))
    )

    results = []
    for idx, score in zip(indices[0], scores[0]):
        if idx < len(df_filtered):
            doc = df_filtered.iloc[idx]["document"]
            results.append((doc, float(score)))

    return results


### 4.3. Query Result

4.3.1. Result 1: [query] 강원도에 있는 해산물 음식점을 찾아줘

In [48]:
query = "강원도에 있는 해산물 음식점을 찾아줘"

conds = parse_query(query)

filtered_df = filter_dataframe(
    df,
    regions=conds["regions"],
    categories=conds["categories"]
)

print("Filtered rows:", len(filtered_df))

Filtered rows: 756


In [51]:
corpus_embeddings = embedder.encode(
    filtered_df["document"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True
)

index = build_faiss_index(corpus_embeddings)

Batches: 100%|██████████| 24/24 [00:06<00:00,  3.83it/s]


In [52]:
results = search(
    query,
    filtered_df,
    embedder,
    index,
    k=5
)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n#{i} (score={score:.4f})")
    print(doc)


#1 (score=0.5855)
가게 이름은 원미매운탕입니다. 강원특별자치도 고성군에 위치한 해산물 구이/찜 매장입니다. 주소는 강원특별자치도 고성군 간성읍 광평길 2입니다.

#2 (score=0.5809)
가게 이름은 설악민물장어입니다. 강원특별자치도 속초시에 위치한 해산물 구이/찜 매장입니다. 주소는 강원특별자치도 속초시 선사로5길 19입니다.

#3 (score=0.5604)
가게 이름은 자연매운탕입니다. 강원특별자치도 평창군에 위치한 해산물 구이/찜 매장입니다. 주소는 강원특별자치도 평창군 봉평면 봉평북로 100-2입니다.

#4 (score=0.5590)
가게 이름은 선창활어횟집입니다. 강원특별자치도 속초시에 위치한 횟집 매장입니다. 주소는 강원특별자치도 속초시 영랑해안길 3입니다.

#5 (score=0.5581)
가게 이름은 흥미횟집입니다. 강원특별자치도 강릉시에 위치한 횟집 매장입니다. 주소는 강원특별자치도 강릉시 강동면 안인진길 29-2입니다.


4.3.2. Result 2: [query] 경남에 있는 호텔을 추천해줘

In [64]:
query = "경남에 있는 호텔을 추천해줘"

conds = parse_query(query)

filtered_df = filter_dataframe(
    df,
    regions=conds["regions"],
    categories=conds["categories"]
)

print("Filtered rows:", len(filtered_df))

Filtered rows: 115


In [65]:
corpus_embeddings = embedder.encode(
    filtered_df["document"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True
)

index = build_faiss_index(corpus_embeddings)

Batches: 100%|██████████| 4/4 [00:01<00:00,  2.12it/s]


In [66]:
results = search(
    query,
    filtered_df,
    embedder,
    index,
    k=5
)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n#{i} (score={score:.4f})")
    print(doc)


#1 (score=0.5285)
가게 이름은 화왕산스파호텔입니다. 경상남도 창녕군에 위치한 호텔/리조트 매장입니다. 주소는 경상남도 창녕군 부곡면 온천1길 3입니다.

#2 (score=0.5087)
가게 이름은 호텔카리브입니다. 경상남도 진주시에 위치한 여관/모텔 매장입니다. 주소는 경상남도 진주시 진양호로 220입니다.

#3 (score=0.5059)
가게 이름은 전망좋은모텔입니다. 경상남도 창원시 진해구에 위치한 여관/모텔 매장입니다. 주소는 경상남도 창원시 진해구 웅천서로 21입니다.

#4 (score=0.4956)
가게 이름은 오아시스호텔입니다. 경상남도 거제시에 위치한 호텔/리조트 매장입니다. 주소는 경상남도 거제시 장평4로2길 11-9입니다.

#5 (score=0.4946)
가게 이름은 에이플러스호텔입니다. 경상남도 거제시에 위치한 여관/모텔 매장입니다. 주소는 경상남도 거제시 고현로10길 6입니다.


================================================================================================================

강남구 카페를 검색하는 것 외의 다른 지역이나 업종을 검색하는 것까지 가능하게 했지만, 경남 호텔에 대한 검색 결과를 보면 '모텔'에 해당하는 매장이 '호텔'보다 상위 문서에 나오고 있는 것을 확인할 수 있다.

가중치 부여로 서브 카테고리 매칭을 적용하면 검색 결과에 도움이 될지 .. 확인해보자.

================================================================================================================

### 4.4 검색 로직 개선

4.4.1. 자연어 쿼리 내 핵심 키워드 활용

사용자가 입력한 구체적인 단어(예: "호텔")를 sub_intent 변수에 저장 -> 사용자가 특정 단어에 부여한 우선순위를 검색 결과에 반영한다.

In [80]:
def parse_query_advanced(query):
    conditions = {"regions": [], "categories": [], "sub_intent": None}
    
    for region, keywords in REGION_KEYWORDS.items():
        if any(kw in query for kw in keywords):
            conditions["regions"].append(region)

    for cat, keywords in CATEGORY_KEYWORDS.items():
        matched_keywords = [kw for kw in keywords if kw in query]
        
        if matched_keywords:
            conditions["categories"].append(cat)
            # 사용자가 입력한 가장 구체적인 단어(예: "호텔")를 sub_intent로 저장
            # 여러 개일 경우 첫 번째 단어를 선택
            conditions["sub_intent"] = matched_keywords[0]
            break
            
    return conditions

4.4.2. 검색 결과가 사용자의 의도대로 나오도록 가중치 부여

벡터 유사도 점수와 데이터의 메타데이터(업종)를 더함 -> 사용자의 의도에 가장 부합하는 매장을 상단에 노출

In [81]:
def search_weighted(query, df_filtered, embedder, index, sub_intent=None, k=5):
    if len(df_filtered) == 0: return []

    query_emb = embedder.encode([query], normalize_embeddings=True)
    scores, indices = index.search(query_emb, min(k * 2, len(df_filtered)))

    results = []
    for idx, score in zip(indices[0], scores[0]):
        row = df_filtered.iloc[idx]
        final_score = float(score)
        
        # 가중치 로직: 사용자의 의도(호텔)와 실제 분류(호텔/리조트)가 일치하면 점수 가산
        if sub_intent and sub_intent in row["상권업종소분류명"]:
            final_score += 0.1  # 가중치 부여

        results.append((row["document"], final_score))

    results = sorted(results, key=lambda x: x[1], reverse=True)
    return results[:k]

4.4.3. Query Result

In [84]:
query = "경남에 있는 호텔을 추천해줘"

conds = parse_query_advanced(query)

filtered_df = filter_dataframe(
    df,
    regions=conds["regions"],
    categories=conds["categories"],
)

print("Filtered rows:", len(filtered_df))

Filtered rows: 115


In [85]:
corpus_embeddings = embedder.encode(
    filtered_df["document"].tolist(),
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True
)

index = build_faiss_index(corpus_embeddings)

Batches: 100%|██████████| 4/4 [00:00<00:00,  5.39it/s]


In [86]:
results = search_weighted(
    query,
    filtered_df,
    embedder,
    index,
    sub_intent=conds["sub_intent"],
    k=5
)

for i, (doc, score) in enumerate(results, 1):
    print(f"\n#{i} (score={score:.4f})")
    print(doc)


#1 (score=0.6285)
가게 이름은 화왕산스파호텔입니다. 경상남도 창녕군에 위치한 호텔/리조트 매장입니다. 주소는 경상남도 창녕군 부곡면 온천1길 3입니다.

#2 (score=0.5956)
가게 이름은 오아시스호텔입니다. 경상남도 거제시에 위치한 호텔/리조트 매장입니다. 주소는 경상남도 거제시 장평4로2길 11-9입니다.

#3 (score=0.5087)
가게 이름은 호텔카리브입니다. 경상남도 진주시에 위치한 여관/모텔 매장입니다. 주소는 경상남도 진주시 진양호로 220입니다.

#4 (score=0.5059)
가게 이름은 전망좋은모텔입니다. 경상남도 창원시 진해구에 위치한 여관/모텔 매장입니다. 주소는 경상남도 창원시 진해구 웅천서로 21입니다.

#5 (score=0.4946)
가게 이름은 에이플러스호텔입니다. 경상남도 거제시에 위치한 여관/모텔 매장입니다. 주소는 경상남도 거제시 고현로10길 6입니다.


🔖

4.3.2의 결과에서는 '여관/모텔'에 해당하는 매장이 '호텔/리조트'보다 더 상위에 랭크되는 결과를 볼 수 있었다.

업종명에 가중치를 부여하는 방식을 적용하여, 사용자의 의도와 일치하는 매장이 더 높은 점수를 받도록 개선했다.

결과적으로 의도한대로 '호텔' 관련 매장이 상위에 랭크되는 것을 확인할 수 있었다.

다만, 현재 이 코드에서는 더 복잡한 쿼리에 대해서 결과가 의도대로 잘 나오지 않는 문제가 발생할 수 있다. 